In [1]:
import utilities as u
import torch
from torch_geometric.data import Data, InMemoryDataset, DataLoader

torch.manual_seed(42)

# Tensorboard Plotting 

In [2]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/problem_estimation_binary_1')

# Define Dataset

In [3]:
from tqdm import tqdm

class TopLevelProofDataset(InMemoryDataset):
    def __init__(self, root='', transform=None, pre_transform=None):
        super(TopLevelProofDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        
    @property
    def raw_file_names(self):
        return []
    
    @property
    def processed_file_names(self):
        return ['../top_level_proofs_binary.dataset']
    
    def download(self):
        pass
    
    def process(self):
        global data
        data_list = []
        
        for thm, y in data:
            thm = u.process_theorem(thm)
            tree, distinct_features = u.thm_to_tree(thm)
            
            x, edge_index = u.graph_to_data(tree, list(distinct_features))
            
            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)
        
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# SAGEConv Layer

In [4]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class SAGEConv(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(SAGEConv, self).__init__(aggr='mean') #  "Max" aggregation.
        self.lin = torch.nn.Linear(in_channels, out_channels)
        self.act = torch.nn.ReLU()
        self.update_lin = torch.nn.Linear(in_channels + out_channels, in_channels, bias=False)
        self.update_act = torch.nn.ReLU()
        
    def forward(self, x, edge_index):
        # x has shape [N, in_channels]
        # edge_index has shape [2, E]
        
        
        edge_index, _ = remove_self_loops(edge_index)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x)

    def message(self, x_j):
        # x_j has shape [E, in_channels]

        x_j = self.lin(x_j)
        x_j = self.act(x_j)
        
        return x_j

    def update(self, aggr_out, x):
        # aggr_out has shape [N, out_channels]


        new_embedding = torch.cat([aggr_out, x], dim=1)
        
        new_embedding = self.update_lin(new_embedding)
        new_embedding = self.update_act(new_embedding)
        
        return new_embedding

# GNN definition

In [5]:
embed_dim = 128
from torch_geometric.nn import TopKPooling, GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
import torch.nn.functional as F
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = GCNConv(embed_dim, 128)
        self.embedding = torch.nn.Embedding(num_embeddings=len(distinct_features)+1, embedding_dim=embed_dim)
        self.pool1 = TopKPooling(128, ratio=0.8)
        self.conv2 = SAGEConv(128, 128)
        self.pool2 = TopKPooling(128, ratio=0.8)
        self.conv3 = SAGEConv(128, 128)
        self.pool3 = TopKPooling(128, ratio=0.8)
        self.lin1 = torch.nn.Linear(256, 128)
        self.lin2 = torch.nn.Linear(128, 64)
        self.lin3 = torch.nn.Linear(64, 2)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.act1 = torch.nn.ReLU()
        self.act2 = torch.nn.ReLU()  
  
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.embedding(x)
        x = x.squeeze(1)      

        x = self.conv1(x, edge_index)
        x = F.relu(x)

        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, None, batch)
        x1 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)
#         x = x1

        x = F.relu(self.conv2(x, edge_index))
     
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, None, batch)
        x2 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = F.relu(self.conv3(x, edge_index))

        x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        x3 = torch.cat([gmp(x, batch), gap(x, batch)], dim=1)

        x = x1 + x2 + x3

        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)      
        x = F.dropout(x, p=0.5, training=self.training)
#         x = self.lin3(x).squeeze(1)

        x = F.log_softmax(self.lin3(x), dim=1).squeeze(1)
#         x = x * 10
#         x = torch.ceil(x)

        return x

# Data inspections

In [6]:
data = u.make_data()
# data = data[0:2]

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590


In [7]:
distinct_features = set(i for i in range(878))

In [8]:


# for idx, (thm, _) in enumerate(data):
#     if idx % 1000 == 0:
#         print(f'{idx} / {len(data)}')
#     thm = u.process_theorem(thm)
#     thm_tree, features = u.thm_to_tree(thm)
#     distinct_features = distinct_features.union(features)

In [9]:
# test_thm = '(fun (a A B) (a A (a A B)))'
# print(test_thm)
# thm = u.process_theorem(test_thm)
# print(thm)
# thm_tree = u.thm_to_tree(thm)
# print(thm_tree)

In [10]:
# counter = dict()
# for _, y in data:
#     if y in counter:
#         counter[y] += 1
#     else:
#         counter[y] = 1
# counter = list(counter.items())
# counter.sort(key=lambda x: x[1], reverse=True)
# percentages = [(x, y/len(datapoints)*100) for x,y in counter]
# percentages

# Create Dataset

In [11]:
from math import floor

dataset = TopLevelProofDataset()
dataset.shuffle()

train_dataset = dataset[:floor(len(dataset)/2)]
# train_dataset = dataset[:2]
# for x in train_dataset:
#     print(x)
valid_dataset = dataset[floor(len(dataset)/2) : 3*floor(len(dataset)/2)]
test_dataset = dataset[3*floor(len(dataset)/2):]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
print(train_dataset)

Processing...
Done!
TopLevelProofDataset(6838)


# Train

In [12]:
batch_size = 64
num_epochs = 500

def train():
    global epoch
    model.train()
    
    loss_all = 0
    correct = 0
    for i, data in enumerate(train_loader):
        x = data.x.squeeze(1)

        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        #print(label, output)
        loss = F.nll_loss(output, label)
        loss.backward()
        loss_all += data.num_graphs * loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(label.data.view_as(pred)).sum()

        optimizer.step()
    
    writer.add_scalar('training loss',
                     loss_all / len(train_dataset),
                     epoch)
    
    writer.add_scalar('training accuracy',
                     correct.item() / len(train_dataset),
                     epoch)
    
    return loss_all / len(train_dataset), correct.item() / len(train_dataset)


def test():
    global epoch
    model.eval()
    
    loss_all = 0
    correct = 0
    for i, data in enumerate(valid_loader):
        x = data.x.squeeze(1)

        data = data.to(device)
        output = model(data)
        label = data.y.to(device)
        loss = F.nll_loss(output, label)
        loss_all += data.num_graphs * loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(label.data.view_as(pred)).sum()
    
    writer.add_scalar('validation loss',
                     loss_all / len(valid_dataset),
                     epoch)
    
    writer.add_scalar('validation accuracy',
                     correct.item() / len(valid_dataset),
                     epoch)
    
    return loss_all / len(valid_dataset), correct.item() / len(valid_dataset)
    
    
device = torch.device('cuda')

model = Net().to(device)
# clip = 5
# torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
crit = torch.nn.CrossEntropyLoss()
train_loader = DataLoader(train_dataset, batch_size=batch_size)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)


for epoch in tqdm(range(num_epochs)):
    epoch_loss, epoch_acc = train()
    #print(f'train - loss: {epoch_loss}, acc: {epoch_acc}')
    
    if epoch % 10 == 9:
        valid_loss, valid_acc = test()
        #print(f'valid - loss: {valid_loss}, acc: {valid_acc}')
    
    #print()
        
    

100%|██████████| 500/500 [46:00<00:00,  5.52s/it]
